## Sentence Embedding

based on https://huggingface.co/LazarusNLP/all-indo-e5-small-v4

In [1]:
#!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer
import numpy as np
import pandas as pd
from sklearn.decomposition import LatentDirichletAllocation

c:\Users\Revalda Putawara\.conda\envs\mbgsentiment\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
RANDOMSTATE=241

In [3]:
#first execution : 6 minutes
#model = SentenceTransformer('LazarusNLP/all-indo-e5-small-v4')

In [3]:
df = pd.read_csv(r"dataset\processed\processed_data_sentiment.csv")

## BERT

In [ ]:
#sentences = ["This is an example sentence", "Each sentence is converted"]
sentences = [df.at[32,"text"],df.at[64,"text"]]
embeddings = model.encode(sentences)
print(embeddings)

In [6]:
df['cs_embeddings'] = df['text_cleaned_stemmed'].apply(lambda x: model.encode(x))

In [ ]:
df.head()

In [8]:
#df.to_pickle(r"dataset/processed/text_embeddings.pkl")
#df = pd.read_pickle(r"dataset/processed/text_embeddings.pkl")

In [ ]:
temp = embeddings[0] - embeddings[1]
distance = np.sqrt(np.sum(np.square(temp)))

print("Euclidean Distance: ", distance)

In [ ]:
pd.DataFrame(df.iloc[:,-1].to_list())

In [ ]:
df[["id"]]

In [12]:
df1 = df[["id"]]
df2 = pd.DataFrame(df.iloc[:,-1].to_list())
df_emb = pd.concat([df1, df2], axis=1)

In [ ]:
df_emb.head()

## LDA

based on https://medium.com/analytics-vidhya/topic-modelling-using-word-embeddings-and-latent-dirichlet-allocation-3494778307bc

In [4]:
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction import text

In [5]:
f = open("id.stopwords.02.01.2016.txt", "r")
stopword_list = []
for line in f:
    stripped_line = line.strip()
    line_list = stripped_line.split()
    stopword_list.append(line_list[0])
f.close()

stopword_list.append("nya")
stopword_list.append("ya")

len(stopword_list)

760

In [6]:
#stopword_list = "ada adalah"

In [7]:
my_stop_words = text.ENGLISH_STOP_WORDS.union(stopword_list)

In [8]:
df.head()

,Unnamed: 0.2,Unnamed: 0.1,id,edit_history_tweet_ids,created_at,text,author_id,Unnamed: 0,text_cleaned,text_cleaned_stemmed,mdhugol_sentiment,mdhugol_sentiment_score,indobert_sentiment,indobert_sentiment_score
0,0,0,1880655687866552831,1880655687866552831,2025-01-18T16:37:11.000Z,Ketua Umum PDIP Megawati Soekarnoputri sempat ...,917707622026928129,NaN,ketua umum pdip megawati soekarnoputri sempat ...,ketua umum pdip megawati soekarnoputri sempat ...,neutral,0.989232,Neutral,0.999553
1,1,1,1880655628819165446,1880655628819165446,2025-01-18T16:36:57.000Z,@HabilIsl Hold $WOWO dapat airdrop makan siang...,112500656,NaN,hold wowo dapat airdrop makansianggratis,hold wowo dapat airdrop makansianggratis,positive,0.912627,Neutral,0.999512
2,2,2,1880655540717842742,1880655540717842742,2025-01-18T16:36:36.000Z,KALAU MBG NYA INI GUE JUGA MAU DUT BIAR GUE MU...,1266052828742602752,NaN,kalau mbg nya ini gue juga mau dut biar gue mu...,kalau mbg nya ini gue juga mau dut biar gue mu...,negative,0.982285,Negative,0.999828
3,3,3,1880655480122651014,1880655480122651014,2025-01-18T16:36:22.000Z,@neohistoria_id program MBG tuh B nya Bersyuku...,1838116873331945472,NaN,program mbg tuh b nya bersyukur bukan bergizi,program mbg tuh b nya syukur bukan gizi,negative,0.985092,Negative,0.996238
4,4,4,1880655463207117137,1880655463207117137,2025-01-18T16:36:17.000Z,@03__nakula Ga semua anak Indonesia bisa makan...,1511016593353936896,NaN,enggak semua anak indonesia bisa makan siang ...,enggak semua anak indonesia bisa makan siang i...,negative,0.990175,Neutral,0.564391


In [9]:
tf_vectorizer = TfidfVectorizer(stop_words=list(my_stop_words), max_features=50000)
tweet_matrix = tf_vectorizer.fit_transform(df['text_cleaned_stemmed'])

C:\Users\Revalda Putawara\AppData\Roaming\Python\Python312\site-packages\sklearn\feature_extraction\text.py:402: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['baiknya', 'berkali', 'kali', 'kurangnya', 'mata', 'olah', 'sekurang', 'setidak', 'tama', 'tidaknya'] not in stop_words.
  warnings.warn(


In [10]:
lda = LatentDirichletAllocation(n_components=5, learning_method='online', 
                                          random_state=RANDOMSTATE, verbose=0, n_jobs = -1)

In [11]:
lda_model = lda.fit(tweet_matrix)
lda_matrix = lda_model.transform(tweet_matrix)
lda_matrix

array([[0.03893116, 0.8439003 , 0.03837805, 0.04064049, 0.03815   ],
       [0.06897738, 0.72317065, 0.06902739, 0.06906648, 0.0697581 ],
       [0.0518288 , 0.05197352, 0.79238007, 0.05208199, 0.05173562],
       ...,
       [0.7596791 , 0.06005814, 0.06004589, 0.06020113, 0.06001574],
       [0.04257712, 0.04196557, 0.04225833, 0.83129004, 0.04190894],
       [0.04151527, 0.83384209, 0.0414249 , 0.04205353, 0.0411642 ]],
      shape=(375, 5))

In [12]:
tf_vectorizer.get_feature_names_out()

array(['01rw', '08surakarta', '0eralatan', ..., 'zhammmmmm', 'zi', 'zone'],
      shape=(1904,), dtype=object)

In [13]:
def print_topics(model, count_vectorizer, n_top_words):
    words = tf_vectorizer.get_feature_names_out()
    for topic_idx, topic in enumerate(model.components_):
      
        print("\nTopic #%d:" % topic_idx )
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))

# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda_model, tweet_matrix, 15)

Topics found via LDA:

Topic #0:
mbg makanbergizigratis program indonesia laksana evaluasi visipresidenprabowo makanbegizigratis miskin kasi rakyat uang mending banding anggar

Topic #1:
makansianggratis mbg gizi anak makan program makanbergizigratis butuh anakanak tanggap anggar udah gua arti ekonomi

Topic #2:
mbg makansianggratis anak program makan gue enak akal duit dana masuk bilang kantin beli budget

Topic #3:
program makanbergizigratis mbg makan bikin makansianggratis gratis rumah sekolah orang prabowo banget presiden emang perintah

Topic #4:
makansianggratis dapet dipake nomer arghhh bingung ril gak mbg kurus enak takut papa gitu king


## KMeans Clustering

In [13]:
from sklearn.cluster import KMeans
from matplotlib import pyplot as plt

In [ ]:
# Loop over values of k from 1 to 10
K=10
inertias_mbg = []

for k in range(1, K+1):
    # Instantiate the KMeans class with k clusters
    ##TODO##
    kmeansiris = KMeans(k)

    # Fit the model to the data
    ##TODO##
    kmeansiris.fit(df2)

    # Store the value of the inertia for this value of k
    ##TODO##
    inertias_mbg.append(kmeansiris.inertia_)

# Plot the elbow
##TODO##
plt.figure()
plt.plot(range(1, K+1), inertias_mbg, 'bx-')
plt.xlabel('k')
plt.ylabel('Inertia')
plt.title('The elbow method showing the optimal k')

In [15]:
k = 5
kmeansmbg = KMeans(k,random_state=RANDOMSTATE)

In [ ]:
kmeansmbg.fit(df2)

In [17]:
df["assigned_cluster"] = kmeansmbg.predict(df2)

In [ ]:
df.head()

In [19]:
df.to_csv(r"dataset\processed\kmeans_result.csv")

In [20]:
df.to_excel(r"dataset\processed\kmeans_result.xlsx")